# Import the necessary libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [2]:
# import the neccessary libraries
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

# Load the data and perform all the necessary pre-processing

In [3]:
# load the data and label in separate lists
numerals_directory = '/content/drive/MyDrive/Dataset/Numerals'
data = []
label = []

for numeral in os.listdir(numerals_directory):
    numerals_path = os.path.join(numerals_directory, numeral)
    for image in os.listdir(numerals_path):
        image_path = os.path.join(numerals_path, image)
        img = load_img(image_path, target_size=(128, 128))
        img = img_to_array(img)
        data.append(img)
        label.append(int(numeral))

# print("-----Completed-----")

In [4]:
# convert the lists to numpy arrays
data = np.array(data)
label = np.array(label)

In [5]:
# check the size of the data and labels
print(data.shape)
print(label.shape)

(9532, 128, 128, 3)
(9532,)


In [6]:
# converting the numeral labels to array like label
label = to_categorical(label, num_classes=10)

In [7]:
# shuffle the data
n = np.arange(9532)
np.random.shuffle(n)
data_shuffled = data[n]
label_shuffled = label[n]

print("---Shuffling Completed Succesfully---")

---Shuffling Completed Succesfully---


In [8]:
data_shuffled = data_shuffled/255.0

In [9]:
# split the data into train and test data
# train-size: 80%
# val-size: 20%
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.1, stratify=label, random_state=12)

# CUSTOM MODEL

In [10]:
# make a sequential model

model = Sequential()

model.add(Conv2D(64, (3,3), activation='relu', input_shape=(128,128,3)))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 63, 63, 64)       256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 63, 63, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0

In [11]:
# compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
# set callbacks
callback = EarlyStopping(
    monitor='loss',
    patience=3
)

In [13]:
# fit the train data into the model
b = model.fit(
    x_train, y_train,
    epochs=40,
    callbacks=[callback],
    batch_size=32
)

Epoch 1/40
269/269 [==============================] - 27s 60ms/step - loss: 0.4936 - accuracy: 0.8621
Epoch 2/40
269/269 [==============================] - 16s 60ms/step - loss: 0.0606 - accuracy: 0.9831
Epoch 3/40
269/269 [==============================] - 16s 59ms/step - loss: 0.0176 - accuracy: 0.9942
Epoch 4/40
269/269 [==============================] - 16s 58ms/step - loss: 0.0386 - accuracy: 0.9888
Epoch 5/40
269/269 [==============================] - 16s 59ms/step - loss: 0.0380 - accuracy: 0.9915
Epoch 6/40
269/269 [==============================] - 16s 59ms/step - loss: 0.0501 - accuracy: 0.9881


In [14]:
len(b.history['loss'])

6

In [15]:
y_pred_custom = model.predict(x_test)

In [16]:
acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_custom, axis=1))
acc*100

96.0167714884696

In [25]:
# save the model
model.save('numeral_model.h5')

# PRE-TRAINED MODEL
## 1. MobileNetV2

In [ ]:
# create the model
# base model
baseModel = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights="imagenet")

# head model
headModel = baseModel.output
headModel = GlobalAveragePooling2D(name="head_pooling")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(10, activation="softmax")(headModel)

# stack the head model on top of the base model
model1 = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
# compile the model
model1.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# set callbacks
callback = EarlyStopping(
    monitor='loss',
    patience=3
)

In [ ]:
c = model1.fit(
    x_train, y_train, epochs=40, callbacks=[callback]
)

Epoch 1/40
269/269 [==============================] - 21s 35ms/step - loss: 0.2965 - accuracy: 0.3123
Epoch 2/40
269/269 [==============================] - 9s 34ms/step - loss: 0.2230 - accuracy: 0.5256
Epoch 3/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1908 - accuracy: 0.6153
Epoch 4/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1722 - accuracy: 0.6629
Epoch 5/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1582 - accuracy: 0.6950
Epoch 6/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1494 - accuracy: 0.7168
Epoch 7/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1443 - accuracy: 0.7276
Epoch 8/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1404 - accuracy: 0.7349
Epoch 9/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1300 - accuracy: 0.7578
Epoch 10/40
269/269 [==============================] - 9s 34ms/step - loss: 0.1250 - accur

In [ ]:
len(c.history['loss'])

28

In [ ]:
y_pred_mobnet = model1.predict(x_test)

In [ ]:
acc_mobnet = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred_mobnet, axis=1))
acc_mobnet*100

84.80083857442348